In [1]:
import sys
if not 'bert_repo' in sys.path:
    sys.path += ['bert_repo']
    
import importlib
import preprocess
importlib.reload(preprocess)
from preprocess import load_data, standard_loader
import model
importlib.reload(model)
from model import SarcasmBertBasic
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from IPython.utils import io

In [2]:
base = "ens_512b6_v%d"
num_ensembles = 12
cfgs = []
SPLIT_RATIO = 0.2
EPOCH = 1.8
dataset = 'twitter'
CONTEXT_LEVEL = 'all'
EMOJ = False

for i in range(1,num_ensembles+1):
    model_name = base % i
    df_model_name = 'model%d' % i
    cfgs.append({
        'model_ver' : model_name,'BERT_MODEL': 'uncased_L-12_H-768_A-12','NUM_TRAIN_EPOCHS': EPOCH,
        'model' : df_model_name, 'state' : i * 10        
    })
print(cfgs)

[{'model_ver': 'ens_512b6_v1', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model1', 'state': 10}, {'model_ver': 'ens_512b6_v2', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model2', 'state': 20}, {'model_ver': 'ens_512b6_v3', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model3', 'state': 30}, {'model_ver': 'ens_512b6_v4', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model4', 'state': 40}, {'model_ver': 'ens_512b6_v5', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model5', 'state': 50}, {'model_ver': 'ens_512b6_v6', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model6', 'state': 60}, {'model_ver': 'ens_512b6_v7', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 'model': 'model7', 'state': 70}, {'model_ver': 'ens_512b6_v8', 'BERT_MODEL': 'uncased_L-12_H-768_A-12', 'NUM_TRAIN_EPOCHS': 1.8, 

In [3]:
final_res = pd.DataFrame()
for cfg in cfgs:
    model = cfg['model']
    state = cfg['state']
    
    train,test = load_data(load_fn=standard_loader(dataset), context_extent=CONTEXT_LEVEL, split=SPLIT_RATIO, random_state=state,emoji_totext=EMOJ)
    m = SarcasmBertBasic(cfg)
    print("Begin Train")
    with io.capture_output() as captured:
        m.fit(train)
    ##################################
    print("Begin Eval")
    with io.capture_output() as captured:
        preds = m.predict(test)

    y_true = [e.label for e in test]
    labels = [ 'NOT_SARCASM', 'SARCASM']
    y_pred = []

    # Dont try and be cute by vectorizing this. It runs out of memory. 
    for p in preds:
        log_probs = p['probabilities']
        probs = [np.exp(e) for e in log_probs]
        lab = labels[np.argmax(probs)]
        y_pred.append(lab) 
    

    acc = accuracy_score(y_true,y_pred)
    prec = precision_score(y_true,y_pred,pos_label='SARCASM')
    rec = recall_score(y_true,y_pred,pos_label='SARCASM')
    f1 = f1_score(y_true,y_pred,pos_label='SARCASM')

    ns_prec = precision_score(y_true,y_pred,pos_label='NOT_SARCASM')
    ns_rec = recall_score(y_true,y_pred,pos_label='NOT_SARCASM')
    ns_f1 = f1_score(y_true,y_pred,pos_label='NOT_SARCASM')

    print( 
        """Accuracy:{}\n\nSARCASM\nPrecision:{}\nRecall:{}\nF1 Score:{}
        \nNOT SARCASM\nPrecision:{}\nRecall:{}\nF1 Score:{}""".format(acc,prec,rec,f1,ns_prec,ns_rec,ns_f1)
    )
    
    
    ###################################
    print("Begin Test Predict")
    live_test = load_data(load_fn=standard_loader('%s_test' % dataset), context_extent=CONTEXT_LEVEL, split=None,emoji_totext=EMOJ)
    live_df = load_data(load_fn=standard_loader('%s_test' % dataset),bert_pair_shape=False, context_extent=CONTEXT_LEVEL,split=None,emoji_totext=EMOJ)

    ids = live_df['id'].tolist()

    with io.capture_output() as captured:
        live_preds = m.predict(live_test)
    labels = [ 'NOT_SARCASM', 'SARCASM']
    #live_y_pred = []
    res = pd.DataFrame()
    # Dont try and be cute by vectorizing this. It runs out of memory. 
    for this_id, p in zip( ids, live_preds):
        log_probs = p['probabilities']
        probs = [np.exp(e) for e in log_probs]
        lab = labels[np.argmax(probs)]
        #live_y_pred.append(lab)    
        res = res.append({'id': this_id, model: lab},ignore_index=True)
    if final_res.empty:
        final_res = res
    else:
        final_res = final_res.merge(res, on='id')


***** Model output directory: bert_out_files/ens_512b6_v1 *****
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Begin Train
INFO:tensorflow:Using config: {'_model_dir': 'bert_out_files/ens_512b6_v1', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff47012ced0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'bert_out_files/ens_512b6_v1', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff47012ced0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Writing example 0 of 4000


...

INFO:tensorflow:Restoring parameters from bert_out_files/ens_512b6_v12/model.ckpt-1200


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [7]:
f = open("answer.txt", "w")
model_list = [ 'model%d' % i for i in range(1,num_ensembles+1)]

final_res['label'] = final_res.loc[:,model_list].mode(axis=1)
for _ind, row in final_res.iterrows():
    f.write('{},{}\n'.format(row['id'],row['label']))
    
f.close()    

ValueError: Wrong number of items passed 2, placement implies 1

In [12]:
#new_final_res = final_res
final_res.head(5)

,id,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12
0,twitter_1,SARCASM,SARCASM,SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,SARCASM,NOT_SARCASM
1,twitter_2,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM
2,twitter_3,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM
3,twitter_4,NOT_SARCASM,NOT_SARCASM,SARCASM,SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,NOT_SARCASM,SARCASM,SARCASM,SARCASM
4,twitter_5,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM,SARCASM


In [15]:
#final_res[model_list].mode(axis=1)
final_res.loc[:,model_list].mode(axis=1)[0]

0       NOT_SARCASM
1           SARCASM
2           SARCASM
3       NOT_SARCASM
4           SARCASM
           ...     
1795    NOT_SARCASM
1796        SARCASM
1797    NOT_SARCASM
1798    NOT_SARCASM
1799    NOT_SARCASM
Name: 0, Length: 1800, dtype: object